In [62]:
from typing import List

import pandas as pd
import numpy as np

In [2]:
DF = pd.DataFrame
S = pd.Series
ND = np.ndarray

In [10]:
np.random.seed(31)

In [29]:
def gen_toy() -> DF:
    records = []
    for x1 in range(5):
        for x2 in range(5):
            for x3 in range(5):
                n_samples = np.random.randint(1000)
                y_avg = np.random.normal()
                records.append(dict(x1=x1, x2=x2, x3=x3, n_samples=n_samples, y_avg=y_avg))
    return pd.DataFrame(records)

In [51]:
dataset = gen_toy()

In [116]:
def lift(data: DF, n_samples: str, driver: str, segments: List[str], target: str, thresh: int):
    data = data.copy()
    
    # driver列を0,1の２クラスへ
    # thresh未満は0、threshより大きいと1
    # n_samplesは合計、targetは加重平均を取る
    tmp = '__tmp__'
    data.loc[:, tmp] = 0
    data.loc[data[driver] > thresh, tmp] = 1
    data = (
        data
        .drop(columns=[driver])
        .rename(columns={tmp: driver})
        .assign(y_prod_sum=data[n_samples] * data[target])
        .groupby(segments + [driver])
        .sum()[[n_samples, 'y_prod_sum']]
        .reset_index()
    )
    data = data.assign(**{target: data.y_prod_sum / data[n_samples]}).drop(columns=['y_prod_sum'])
    
    # driver列の0と1の間のtargetの差がそのセグメントのliftとなる
    pivot = data.pivot(index=segments, columns=driver, values=target)
    pivot.columns.name = None
    lift = pivot.assign(lift=pivot.loc[:, 1] - pivot.loc[:, 0])['lift']
    
    # セグメントごとのサンプル数
    size = data.groupby(segments).sum()[n_samples]
    
    # 加重平均をとる
    return size.multiply(lift).sum() / size.sum()

In [117]:
lift(
    dataset,
    driver='x3',
    segments=['x1', 'x2'],
    n_samples='n_samples',
    target='y_avg',
    thresh=2.5
)

0.3243240764876645